# 9. 머신러닝 프로세스6 - 다중분류


예측 결과가 투표여부의 이진분류가 아닌 **지지정당등과 같이 여러항에 해당하는 다중 분류 처리방법**에 대해 알아 보겠습니다.

In [22]:
# 경고레벨 조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data=pd.read_csv('./extrafiles/Fvote.csv', encoding='utf-8')

# 독립변수와 종속 변수 분리
X = data[data.columns[1:13]]
y = data[['parties']]

# y 값의 종류와 갯수
print(y.value_counts())

# 훈련 및 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

parties
4          83
2          53
1          50
3          25
dtype: int64


In [10]:
# LogisticRegression 적용
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [11]:
# 훈련데이터 예측값 도출
pred_train = model.predict(X_train)
model.score(X_train, y_train)

0.6139240506329114

In [13]:
# 테스트데이터 예측값 도출
pred_test = model.predict(X_test)
model.score(X_test, y_test)

0.5283018867924528

![다중분류 혼동행렬](./extrafiles/multimatrix.png)

In [34]:
# 다중 분류의 혼동행렬 산출
from sklearn.metrics import confusion_matrix

# 산출결과값
print(pd.DataFrame(pred_train).value_counts().sort_index())

# 산출결과와 실제값과 비교
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train) # 1번 부터 순서대로 출력됨

1    35
2    37
3    11
4    75
dtype: int64
훈련데이터 오차행렬 : 
 [[21  2  3 11]
 [ 1 25  2 12]
 [ 6  2  5  6]
 [ 7  8  1 46]]


In [26]:
# 테스트데이터 오차행렬
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[ 6  1  2  4]
 [ 1  9  1  2]
 [ 1  2  1  2]
 [ 2  5  2 12]]


In [43]:
# 그리드 서치. 하이퍼 파라미터를 유형별로 다 테스트 해 본후 가장 좋은 것을 추출한다.
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

# 훈련데이터 파라미터 탐색 결과
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Cross-validity Score : {:.3f}".format(grid_search.best_score_))

# 테스트데이터 모델 적용 결과
print("Test set Score : {:.3f}".format(grid_search.score(X_test, y_test)))

Best Parameter : {'C': 0.1}
Best Cross-validity Score : 0.544
Test set Score : 0.585


In [47]:
# 랜덤탐색, 하이퍼 파라미터를 정해진 구간내에서 자유롭게 테스트 하여 가장 좋은 것을 추출한다.
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {'C' : randint(low=0.001, high=100)}

# 랜덤 탐색 기준 생성
random_search = RandomizedSearchCV(LogisticRegression(), param_distributions = param_distribs, cv=5, return_train_score=True)

# 기준 데이터 지정
random_search.fit(X_train, y_train)

# 훈련데이터 파라미터 탐색 결과
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Cross-validity Score : {:.3f}".format(random_search.best_score_))

# 테스트데이터 모델 적용 결과
print("Test set Score : {:.3f}".format(random_search.score(X_test, y_test)))

Best Parameter : {'C': 21}
Best Cross-validity Score : 0.525
Test set Score : 0.509


### Q. 여기서 C 값이란?
정규화 강도 지수 작을수록 정규화를 강하게 한다.
![C벨류](./extrafiles/cparam.jpg)
